<a href="https://www.kaggle.com/code/codewithdark/dataset-dataloader-in-pytorch?scriptVersionId=213032297" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
!pip install torch

In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
from sklearn.datasets import make_classification

X, y = make_classification(
    n_samples=10,       # Number of samples
    n_features=2,       # Number of features
    n_informative=2,    # Number of informative features
    n_redundant=0,      # Number of redundant features
    n_classes=2,        # Number of classes
    random_state=42     # For reproducibility
)


In [4]:
X

array([[ 1.06833894, -0.97007347],
       [-1.14021544, -0.83879234],
       [-2.8953973 ,  1.97686236],
       [-0.72063436, -0.96059253],
       [-1.96287438, -0.99225135],
       [-0.9382051 , -0.54304815],
       [ 1.72725924, -1.18582677],
       [ 1.77736657,  1.51157598],
       [ 1.89969252,  0.83444483],
       [-0.58723065, -1.97171753]])

In [5]:
y

array([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [6]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.long)

In [7]:
X

tensor([[ 1.0683, -0.9701],
        [-1.1402, -0.8388],
        [-2.8954,  1.9769],
        [-0.7206, -0.9606],
        [-1.9629, -0.9923],
        [-0.9382, -0.5430],
        [ 1.7273, -1.1858],
        [ 1.7774,  1.5116],
        [ 1.8997,  0.8344],
        [-0.5872, -1.9717]])

In [8]:
y

tensor([1, 0, 0, 0, 0, 1, 1, 1, 1, 0])

In [9]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]



In [10]:
dataset = CustomDataset(X, y)

In [11]:
len(dataset)

10

In [12]:
dataset[0]

(tensor([ 1.0683, -0.9701]), tensor(1))

In [13]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

for batch_data, batch_labels in dataloader:
    print(batch_data)
    print(batch_labels)

tensor([[-1.1402, -0.8388],
        [-1.9629, -0.9923]])
tensor([0, 0])
tensor([[-0.5872, -1.9717],
        [ 1.0683, -0.9701]])
tensor([0, 1])
tensor([[-0.7206, -0.9606],
        [-2.8954,  1.9769]])
tensor([0, 0])
tensor([[1.8997, 0.8344],
        [1.7774, 1.5116]])
tensor([1, 1])
tensor([[ 1.7273, -1.1858],
        [-0.9382, -0.5430]])
tensor([1, 1])


In [14]:
# prompt: create a NN class to train model on above data

import torch
import torch.nn as nn
import torch.optim as optim

class NN(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [15]:
# Initialize the model, loss function, and optimizer
input_size = 2
hidden_size = 5
num_classes = 2
model = NN(input_size, hidden_size, num_classes)

criterion = nn.CrossEntropyLoss()  # Use CrossEntropyLoss for classification
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 100

for epoch in range(num_epochs):
  for batch_data, batch_labels in dataloader:
    # Forward pass
    outputs = model(batch_data)
    loss = criterion(outputs, batch_labels)

    # Backward and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if (epoch+1) % 10 == 0:
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

Epoch [10/100], Loss: 0.3015
Epoch [20/100], Loss: 0.1004
Epoch [30/100], Loss: 0.0088
Epoch [40/100], Loss: 0.0884
Epoch [50/100], Loss: 0.1946
Epoch [60/100], Loss: 0.0787
Epoch [70/100], Loss: 0.0658
Epoch [80/100], Loss: 0.2090
Epoch [90/100], Loss: 0.7135
Epoch [100/100], Loss: 0.0011


##Now improve the previous code for breast Cancer

In [16]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [17]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [18]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

In [20]:
standard_scaler = StandardScaler()
X_train = standard_scaler.fit_transform(X_train)
X_test = standard_scaler.transform(X_test)

In [21]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.transform(y_test)

In [22]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

In [23]:
train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)

In [24]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [25]:
class BearstCancerNN(nn.Module):
    def __init__(self, num_feature):
        super().__init__()
        self.fc1 = nn.Linear(num_feature, 1) # num_feature should be the number of features (30)
        self.sigmoid  = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        x = self.sigmoid(x)
        return x

In [26]:
lr = 0.1
epochs = 25

In [27]:
model = BearstCancerNN(X_train.shape[1])
loss_function = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=lr)


In [28]:
for i in range(epochs):
    for batch_data, batch_labels in train_dataloader:
        # Forward pass
        outputs = model(batch_data)
        loss = loss_function(outputs, batch_labels.unsqueeze(1).float())

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f'Epoch [{i+1}/{epochs}], Loss: {loss.item():.4f}')



Epoch [1/25], Loss: 0.1764
Epoch [2/25], Loss: 0.0372
Epoch [3/25], Loss: 0.1178
Epoch [4/25], Loss: 0.1206
Epoch [5/25], Loss: 0.0700
Epoch [6/25], Loss: 0.0756
Epoch [7/25], Loss: 0.0657
Epoch [8/25], Loss: 0.0253
Epoch [9/25], Loss: 0.2064
Epoch [10/25], Loss: 0.0820
Epoch [11/25], Loss: 0.0290
Epoch [12/25], Loss: 0.1212
Epoch [13/25], Loss: 0.0369
Epoch [14/25], Loss: 0.0173
Epoch [15/25], Loss: 0.0884
Epoch [16/25], Loss: 0.0176
Epoch [17/25], Loss: 0.0198
Epoch [18/25], Loss: 0.0238
Epoch [19/25], Loss: 0.0327
Epoch [20/25], Loss: 0.1298
Epoch [21/25], Loss: 0.0766
Epoch [22/25], Loss: 0.1374
Epoch [23/25], Loss: 0.0539
Epoch [24/25], Loss: 0.0095
Epoch [25/25], Loss: 0.0998


In [29]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    for batch_data, batch_labels in test_dataloader:
        outputs = model(batch_data)
        predicted = (outputs > 0.5).squeeze().long()

        total += batch_labels.size(0)
        correct += (predicted == batch_labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

Test Accuracy: 98.25%
